In [5]:
#Tache 1
import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
#normalisation
x_train = x_train / 255.0
x_test = x_test / 255.0
# Redimensionnement
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)



In [16]:
from keras.models import Sequential
from keras.layers import Input, Conv2D, Flatten, Dense, Dropout

# Tâche 2 
def get_model_cnn_mnist(input_shape=(28, 28, 1)):
    model = Sequential()

    # entrée
    model.add(Input(shape=input_shape))

    # 2 layers Conv2D, sans les couches convolutives, on a 104938 paramètres, avec on a 1179776 paramètres.
    model.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
    model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))

    # flat
    model.add(Flatten())

    # 2 layers Dense
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    # Output layer
    model.add(Dense(10, activation='softmax'))  

    return model




# résumé
model_cnn_mnist = get_model_cnn_mnist()
model_cnn_mnist.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 8)         80        
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 16)        1168      
                                                                 
 flatten_5 (Flatten)         (None, 9216)              0         
                                                                 
 dense_15 (Dense)            (None, 128)               1179776   
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 32)                4128      
                                                                 
 dropout_11 (Dropout)        (None, 32)               

In [9]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# tache 3
def train_model(model, x_train, y_train, x_test, y_test, epochs=20, batch_size=512):
    model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

    # callback
    model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=True)

    # Entraînement du modèle
    history = model.fit(x_train, y_train,epochs=epochs,batch_size=batch_size,validation_data=(x_test, y_test),callbacks=[model_checkpoint])

    return history

# entrainement
history = train_model(model_cnn_mnist, x_train, y_train, x_test, y_test)
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])


Epoch 1/20
118/118 [==============================] - 10s 71ms/step - loss: 0.0137 - accuracy: 0.9958 - val_loss: 0.0564 - val_accuracy: 0.9870
Epoch 2/20
118/118 [==============================] - 10s 81ms/step - loss: 0.0147 - accuracy: 0.9954 - val_loss: 0.0483 - val_accuracy: 0.9893
Epoch 3/20
118/118 [==============================] - 9s 75ms/step - loss: 0.0125 - accuracy: 0.9962 - val_loss: 0.0532 - val_accuracy: 0.9879
Epoch 4/20
118/118 [==============================] - 9s 73ms/step - loss: 0.0114 - accuracy: 0.9965 - val_loss: 0.0498 - val_accuracy: 0.9887
Epoch 5/20
118/118 [==============================] - 9s 73ms/step - loss: 0.0108 - accuracy: 0.9962 - val_loss: 0.0550 - val_accuracy: 0.9875
Epoch 6/20
118/118 [==============================] - 9s 74ms/step - loss: 0.0113 - accuracy: 0.9964 - val_loss: 0.0491 - val_accuracy: 0.9892
Epoch 7/20
118/118 [==============================] - 8s 67ms/step - loss: 0.0090 - accuracy: 0.9971 - val_loss: 0.0541 - val_accuracy: 0.98

In [14]:
# Questions
# Quels sont les performances de ce modèle vis-à-vis du précédent ?
# Avec un train accuracy de 0.9983 et une validation accuracy de 0.9893, par rapport au modèle précédent qui à une train accuracy de 0.9964 et une validation accuracy de 0.9757, le modèle actuel est plus efficace.
#
# Que pouvez vous souligner sur les performances de ce modèle relativement à son nombre de paramètres ?
# Bien que ce nouveau modèle soit 0.2% plus efficace, il contient 1185482 paramètres, soit 11.29 fois plus que le précédent, et est beaucoup plus lent. Il peut être judicieux de vouloir l'alléger.
#
#Quelle couche du réseau implique beaucoup de paramètres et pourquoi ?
#
# La couche qui contient le plus de  paramètres est la couche dense_9 (1179776). La formule de calcul du nombre de paramètres pour une couche dense est : output_size(ici 128) * ((output_size précédent + 1) ici(9216+1))
assert 128 * (9216 + 1) == 1179776

In [17]:
from keras.layers import MaxPooling2D
from keras.callbacks import TensorBoard
import os

# Tâche 4 
def get_model_cnn_mnist_v2(input_shape=(28, 28, 1)):
    model = Sequential()

    # Layer d'entrée
    model.add(Input(shape=input_shape))

    # 2 layers Conv2D
    model.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))  # Ajout d'une couche de pooling
    model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))  # Ajout d'une autre couche de pooling

    # Flatten
    model.add(Flatten())

    # 2 layers Dense
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    # Output layer
    model.add(Dense(10, activation='softmax'))

    return model


model_cnn_mnist_v2 = get_model_cnn_mnist_v2()

# TensorBoard
log_dir = 'logs/cnn_mnist_with_pooling'
os.makedirs(log_dir, exist_ok=True)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
model_cnn_mnist_v2.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 13, 13, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 16)        1168      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 400)               0         
                                                                 
 dense_18 (Dense)            (None, 128)              

In [ ]:
#Voila la commande utilisée pour charger le tensorboard: tensorboard --logdir logs

In [18]:
from keras.layers import BatchNormalization

# Fonction pour obtenir un modèle avec pooling, dropout et batch normalization
def get_model_with_dropout_and_bn(input_shape=(28, 28, 1)):
    model = Sequential()

    # Layer d'entrée
    model.add(Input(shape=input_shape))

    #  Couches convolutives   avec pooling, dropout batch normalization puis dropout
    model.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    # Flatten
    model.add(Flatten())

    # Couches Dense avec dropout et batch normalization
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(10, activation='softmax'))

    return model

# Affichage du résumé du modèle
model_with_dropout_bn = get_model_with_dropout_and_bn()
model_with_dropout_bn.summary()

# Tensorboard
log_dir = 'logs/cnn_mnist_with_dropout_bn'
os.makedirs(log_dir, exist_ok=True)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 13, 8)         0         
 g2D)                                                            
                                                                 
 batch_normalization (Batch  (None, 13, 13, 8)         32        
 Normalization)                                                  
                                                                 
 dropout_14 (Dropout)        (None, 13, 13, 8)         0         
                                                                 
 conv2d_13 (Conv2D)          (None, 11, 11, 16)        1168      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 5, 16)         

In [21]:
history = train_model(model_cnn_mnist, x_train, y_train, x_test, y_test)
print("Train Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])

Epoch 1/20
118/118 [==============================] - 12s 88ms/step - loss: 0.5247 - accuracy: 0.8371 - val_loss: 0.1291 - val_accuracy: 0.9613
Epoch 2/20
118/118 [==============================] - 7s 62ms/step - loss: 0.1571 - accuracy: 0.9551 - val_loss: 0.0718 - val_accuracy: 0.9776
Epoch 3/20
118/118 [==============================] - 7s 63ms/step - loss: 0.1051 - accuracy: 0.9702 - val_loss: 0.0501 - val_accuracy: 0.9842
Epoch 4/20
118/118 [==============================] - 8s 66ms/step - loss: 0.0763 - accuracy: 0.9778 - val_loss: 0.0500 - val_accuracy: 0.9846
Epoch 5/20
118/118 [==============================] - 7s 63ms/step - loss: 0.0643 - accuracy: 0.9816 - val_loss: 0.0406 - val_accuracy: 0.9872
Epoch 6/20
118/118 [==============================] - 9s 73ms/step - loss: 0.0510 - accuracy: 0.9848 - val_loss: 0.0422 - val_accuracy: 0.9868
Epoch 7/20
118/118 [==============================] - 8s 66ms/step - loss: 0.0427 - accuracy: 0.9871 - val_loss: 0.0390 - val_accuracy: 0.988

In [ ]:
# Questions
# Quelles améliorations ont été apportées aux deux dernières versions de modèles ?
# on a d'abord allégé le modèle grace au pooling, puis après normalisation des données récupérées (application loi normale), on a ajouté un dropout, qui supprime certains neurones au hasard, ce qui permet de limiter le surapprentissage (limitation de l'utilisation trop fréquente de certains chemins) et d'améliorer l'apprentissage
#
#Quel est le principe d’ajout de dropout après une couche de pooling dans le cadre de l’apprentissage d’un modèle ? Cela apporte t-il une meilleure performance dans notre cas ?
#
#Le dropout donne un pourcentage de chance a chaque neurone d'etre désactivé le temps d'un passage (epoch). Le faire après un pooling permet une filtration des données plus importantes, et surtout de faire jouer le hasard de manière plus importante lors de chaque epoch. C'est tres utile dans notre cas, pour éviter le surapprentissage qui était un risque ici notamment. en effet, l'accuracy de la validation est plus élevé ici que sur le premier modèle.